## ClimateBERT to classify filtered english posts into Climate Related and Non-Climate Related
### Input: English language posts (en-US, UK, Canada etc.) 
### Output: Label, Score, and Metadata

In [1]:
pip install datasets -q

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from datasets import load_dataset
from tqdm.auto import tqdm
import json

/Users/tobiasmichelsen/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

json_path = "../data/filtered/en_posts_excluding_emojis.json" #EXCLUDING EMOJIS

dataset = load_dataset("json", data_files=json_path, split="train")

dataset_name = "climatebert/climate_detection"
model_name = "climatebert/distilroberta-base-climate-detector"


model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

results = []

for i , out in enumerate(tqdm(pipe(KeyDataset(dataset,"text",), padding=True, truncation=True))):
   item = dataset[i]
   results.append({
        "repo": item.get("repo"),
        "seq": item.get("seq"),
        "text": item.get("text"),
        "timestamp": item.get("timestamp"),
        "cid": item.get("cid"),
        "uri": item.get("uri"),
        
        "label": out["label"],
        "score": out["score"]
        
    })

with open("../data/processed/Climate_labeled_posts.json", "w") as f:
    json.dump(results, f, indent=2)
    

Device set to use mps:0
100%|██████████| 138261/138261 [4:45:21<00:00,  8.08it/s]  
